In [26]:
from utils import PkdbModel,measurement_type_data, filter_out, convert_unit
from collections import namedtuple
import pandas as pd
from IPython.display import display, HTML

def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell    '''
    return HTML(
         '<table><tr style="background-color:white;">' +
         ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
         '</tr></table>'
     )

In [27]:
def codeine_n_idx(data):
    return (data.substance_intervention == 'codeine phosphate') \
           & (data.substance == 'codeine') \
           & (data['tissue'] == 'plasma')

In [42]:
def all_codeine_idx(data):
     return ((data.substance_intervention == 'codeine phosphate') \
           |(data.substance_intervention == 'codeine sulphate') \
           |(data.substance_intervention == 'codeine')) \
           & (data["substance"] == 'codeine') \
           & (data['tissue'] == 'plasma')

In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.to_numeric()
complete_codeine = all_complete.data.loc[all_codeine_idx(all_complete.data)]
complete_codeine["inferred"] = False

/home/janek/.virtualenvs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [47]:
complete_codeine


,Unnamed: 0,study,output_type,pk,measurement_type,allowed_users,interventions,raw_pk,access,substance,...,"(lean body mass, mean)","(lean body mass, sd)","(lean body mass, unit)","(cyp2d6 variant, choice)","(cyp2d6 variant, count)","(fat weight, count)","(fat weight, mean)","(fat weight, se)","(fat weight, unit)",inferred
3304,3304,Loetsch2006,outputs,21481,clearance,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21473,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3305,3305,Loetsch2006,outputs,21482,cmax,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21474,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3306,3306,Loetsch2006,outputs,21485,vd,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21477,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3307,3307,Loetsch2006,outputs,21486,tmax,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21478,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3327,3327,Loetsch2006,outputs,21483,kel,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21475,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3340,3340,Loetsch2006,outputs,21480,auc_inf,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21472,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3348,3348,Loetsch2006,outputs,21479,auc_end,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21471,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3349,3349,Loetsch2006,outputs,21484,thalf,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21476,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3377,3377,Loetsch2006,timecourses,1845,concentration,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,1779,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3383,3383,Loetsch2006,outputs,21329,vd,"[{'username': 'dimitra'}, {'username': 'mkoeni...",796,21321,public,codeine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [48]:
Inference = namedtuple("Inference",["name","filter_out","measurement_type"])

In [49]:
inferences = [
    Inference(
        name="codeine_clearance",
        filter_out= {"unit":[ 'microgram * milliliter / hour / micromole']},
        measurement_type = "clearance",
    ),
    Inference(
        name="codeine_timecourse",
        filter_out= {"unit":["mole / liter"]},
        measurement_type = "concentration"
    ),
    Inference(
        name="codeine_thalf",
        filter_out= {},
        measurement_type = "thalf"
    ), 
    Inference(
        name="codeine_tmax",
        filter_out= {},
        measurement_type = "tmax"
    ),
    Inference(
        name="codeine_vd",
        filter_out= {"unit":[ 'liter * milligram / millimole']},
        measurement_type = "vd"
    ),
     Inference(
        name="codeine_auc_inf",
        filter_out= {},
        measurement_type = "auc_inf"
    ),
     Inference(
        name="codeine_auc_end",
        filter_out= {},
        measurement_type = "auc_end"
    ),
    Inference(
        name="codeine_kel",
        filter_out= {},
        measurement_type = "kel"
    ),
]

In [50]:
for inference in inferences:
    print(inference.name)
    data = measurement_type_data(complete_codeine,inference.measurement_type)
    data_class = PkdbModel(inference.name,destination="3-inferred")
    data_class.data = data
    [data_class.filter_out(k,v)for k,v in inference.filter_out.items()]
    data_class.infer_from_interventions()
    data_class.infer_from_outputs()
    data_class.save()

    t1 = pd.DataFrame(data_class.data.groupby(["unit", "unit_intervention"]).apply(len), columns=["count"])
    t2 = pd.DataFrame(data_class.data.groupby(["inferred"]).apply(len), columns=["count"])
    t3 = pd.DataFrame(data_class.data.groupby(["calculated"]).apply(len), columns=["count"])
    display(multi_table([t1,t2,t3]))

codeine_clearance


codeine_timecourse


codeine_thalf


codeine_tmax


codeine_vd


codeine_auc_inf


codeine_auc_end


codeine_kel
